In [1]:
# basic imports
import pandas as pd

In [2]:
# create a dataframe of each team's statistics
df_stats = pd.DataFrame()

for year in range(2003, 2023):
    if year == 2020 or year == 2021:
        continue
    # load the previously generated csv files
    df_tour = pd.read_csv(f"stats/{year}-tournament-stats.csv")
    df_total = pd.read_csv(f"stats/{year}-total-stats.csv")
    df_records = pd.read_csv(f"records/{year}-records.csv")
    # drop the year column to prevent repitition
    df_total = df_total.drop("year", axis=1)
    df_tour = df_tour.drop("year", axis=1)
    # merge the records and stats of tournament teams
    df1 = pd.merge(df_records, df_total, on="team", how="left")
    df2 = pd.merge(df_records, df_tour, on="team", how="left")
    # subtract the tournament stats from the total stats so there is no data-leakage
    for col in ['pts', 'fgm', 'fga', '3pm', '3pa', 'ftm', 'fta', 'orb', 'drb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']:
        df1[col] = df1[col] - df2[col]
    
    # concatenate the year to the rest of the dataframe
    df_stats = pd.concat([df_stats, df1], ignore_index=True)

In [3]:
# function to generate the stat per game columns
def stats_per_game(df):
    num_games = df_stats["wins"] + df_stats["losses"]
    # columns to be averaged per game
    total_cols = ['pts', 'orb', 'drb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']
    # append pg to the column names above
    per_game_cols = [c + "pg" for c in total_cols]
    for stat, stat_per_game in zip(total_cols, per_game_cols):
        # add the per game columns to the dataframe
        df_stats.insert(df_stats.columns.get_loc(stat)+1, stat_per_game, df_stats[stat]/num_games)
stats_per_game(df_stats)

In [4]:
# insert the win percentage column
num_games = df_stats["wins"] + df_stats["losses"]
df_stats.insert(df_stats.columns.get_loc("losses")+1, "winp", df_stats["wins"]/num_games)

In [5]:
# insert the conference win percentage column
num_cgames = df_stats["cwins"] + df_stats["closses"]
df_stats.insert(df_stats.columns.get_loc("closses")+1, "cwinp", df_stats["cwins"]/num_cgames)

In [6]:
# insert the field goal percentage column
df_stats.insert(df_stats.columns.get_loc("fga")+1, "fgp", df_stats["fgm"]/df_stats["fga"])

In [7]:
# insert the 3 point percentage column
df_stats.insert(df_stats.columns.get_loc("3pa")+1, "3pp", df_stats["3pm"]/df_stats["3pa"])

In [8]:
# insert the free throw percentage column
df_stats.insert(df_stats.columns.get_loc("fta")+1, "ftp", df_stats["ftm"]/df_stats["fta"])

In [9]:
# round everything in the dataframe to 3 decimal places
df_stats = df_stats.round(3)

In [10]:
df_stats

,year,team,seed,wins,losses,winp,cwins,closses,cwinp,pts,...,ast,astpg,stl,stlpg,blk,blkpg,tov,tovpg,pf,pfpg
0,2003,Oklahoma,1,24,6,0.800,12,4,0.750,2135.0,...,425.0,14.167,208.0,6.933,113.0,3.767,353.0,11.767,558.0,18.600
1,2003,Kentucky,1,28,4,0.875,16,0,1.000,2481.0,...,519.0,16.219,248.0,7.750,166.0,5.188,447.0,13.969,557.0,17.406
2,2003,Texas,1,22,6,0.786,13,3,0.812,2208.0,...,406.0,14.500,179.0,6.393,108.0,3.857,375.0,13.393,570.0,20.357
3,2003,Arizona,1,26,2,0.929,17,1,0.944,2386.0,...,493.0,17.607,240.0,8.571,118.0,4.214,412.0,14.714,497.0,17.750
4,2003,Wake Forest,2,23,6,0.793,12,4,0.750,2274.0,...,423.0,14.586,186.0,6.414,130.0,4.483,431.0,14.862,534.0,18.414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2022,Texas A&M-CC,16,23,11,0.676,7,7,0.500,2615.0,...,511.0,15.029,289.0,8.500,59.0,1.735,483.0,14.206,689.0,20.265
1196,2022,Bryant,16,22,9,0.710,15,2,0.882,2415.0,...,437.0,14.097,196.0,6.323,138.0,4.452,431.0,13.903,520.0,16.774
1197,2022,Wright State,16,21,13,0.618,15,7,0.682,2566.0,...,474.0,13.941,196.0,5.765,100.0,2.941,418.0,12.294,489.0,14.382
1198,2022,Texas Southern,16,18,12,0.600,13,5,0.722,2077.0,...,325.0,10.833,168.0,5.600,150.0,5.000,445.0,14.833,533.0,17.767


In [11]:
# create a dataframe of each tournament game
df_games = pd.DataFrame()
for year in range(2003, 2023):
    if year == 2020 or year == 2021:
        continue
    # load the previously generated csv files
    df_bracket = pd.read_csv(f"brackets/{year}-bracket.csv")
    df_bracket = df_bracket[df_bracket["round"] > 0]
    df_games = pd.concat([df_games, df_bracket], ignore_index=True)
df_games["year"] = df_games["year"].astype(int)

In [12]:
df_games

,year,team0,team1,winner,loser,region,round
0,2003,Kentucky,IUPUI,Kentucky,IUPUI,Midwest Regional,1.0
1,2003,Oregon,Utah,Utah,Oregon,Midwest Regional,1.0
2,2003,Wisconsin,Weber State,Wisconsin,Weber State,Midwest Regional,1.0
3,2003,Dayton,Tulsa,Tulsa,Dayton,Midwest Regional,1.0
4,2003,Missouri,Southern Illinois,Missouri,Southern Illinois,Midwest Regional,1.0
...,...,...,...,...,...,...,...
1129,2022,Miami (FL),Iowa State,Miami (FL),Iowa State,Midwest Regional,3.0
1130,2022,Kansas,Miami (FL),Kansas,Miami (FL),Midwest Regional,4.0
1131,2022,Duke,North Carolina,North Carolina,Duke,none,5.0
1132,2022,Kansas,Villanova,Kansas,Villanova,none,5.0


In [13]:
df = pd.DataFrame()

for i, game in df_games.iterrows():
    winner = df_stats[(df_stats["team"] == game["winner"]) & (df_stats["year"] == game["year"])]
    loser = df_stats[(df_stats["year"] == game["year"]) & (df_stats["team"] == game["loser"])]
    winner_loser = pd.Series(dtype=object)
    winner_loser["year"] = game["year"]
    winner_loser["team0"] = game["winner"]
    winner_loser["team1"] = game["loser"]
    winner_loser = winner_loser.append(winner.squeeze()[2:] - loser.squeeze()[2:])
    winner_loser["result"] = 0
    df = pd.concat([df, winner_loser.to_frame().T], ignore_index=True)
    loser_winner = pd.Series(dtype=object)
    loser_winner["year"] = game["year"]
    loser_winner["team0"] = game["loser"]
    loser_winner["team1"] = game["winner"]
    loser_winner = loser_winner.append(loser.squeeze()[2:] - winner.squeeze()[2:])
    loser_winner["result"] = 1
    df = pd.concat([df, loser_winner.to_frame().T], ignore_index=True)
    
df

,year,team0,team1,seed,wins,losses,winp,cwins,closses,cwinp,...,astpg,stl,stlpg,blk,blkpg,tov,tovpg,pf,pfpg,result
0,2003,Kentucky,IUPUI,-15,8,-9,0.269,6,-4,0.286,...,2.886,-34.0,-0.795,106.0,3.37,-71.0,-1.728,-84.0,-2.018,0
1,2003,IUPUI,Kentucky,15,-8,9,-0.269,-6,4,-0.286,...,-2.886,34.0,0.795,-106.0,-3.37,71.0,1.728,84.0,2.018,1
2,2003,Utah,Oregon,1,1,-2,0.055,1,-5,0.23,...,-4.211,-108.0,-3.225,29.0,1.025,-100.0,-2.738,-53.0,-1.034,0
3,2003,Oregon,Utah,-1,-1,2,-0.055,-1,5,-0.23,...,4.211,108.0,3.225,-29.0,-1.025,100.0,2.738,53.0,1.034,1
4,2003,Wisconsin,Weber State,-7,-4,2,-0.08,-2,4,-0.25,...,-0.918,-4.0,0.323,58.0,2.054,-88.0,-2.165,-151.0,-3.914,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,2022,Duke,North Carolina,-6,4,-3,0.097,1,-1,0.05,...,2.094,42.0,1.077,62.0,1.711,-36.0,-1.401,-10.0,-0.717,1
2264,2022,Kansas,Villanova,-1,2,-1,0.036,-2,0,-0.022,...,3.38,12.0,0.17,67.0,1.905,96.0,2.529,36.0,0.611,0
2265,2022,Villanova,Kansas,1,-2,1,-0.036,2,0,0.022,...,-3.38,-12.0,-0.17,-67.0,-1.905,-96.0,-2.529,-36.0,-0.611,1
2266,2022,Kansas,North Carolina,-7,4,-3,0.097,-1,-1,0.028,...,0.623,40.0,1.018,15.0,0.329,42.0,0.893,65.0,1.489,0


In [14]:
# save the dataframe to a csv file
df.to_csv('feature-vectors.csv', index=False) 